Load and define all the necessary modules and models for this exercise

In [ ]:
# make sure, the numpy library only uses one thread
import os
os.environ["OMP_NUM_THREADS"]="1"

# import libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.interpolate

# import models
from earm.lopez_direct import model as md
from earm.lopez_indirect import model as mi
from earm.lopez_embedded import model as me
from pysb.simulator import ScipyOdeSimulator

In [ ]:
# load experimental data
exp_data = pd.read_csv('EC-RP_IMS-RP_IC-RP_data_for_models.csv', index_col=False)

# read time points from data file
tspan = exp_data['# Time'].values.copy()

# use the rate parameters that are hardcoded into the model
param_values_i = np.array([p.value for p in mi.parameters])
# TODO
# TODO

# arguments for the model simulation (solve the ODEs)
args = {'integrator': 'lsoda', 'use_analytic_jacobian': True}

# simulate the models
solver_i = ScipyOdeSimulator(mi, tspan, **args)
# TODO
# TODO

In [ ]:
# save the trajectories of the ODE integration for analysis
traj_i = solver_i.run(param_values=param_values_i)
# TODO 
# TODO

# normalize the trajectories to the initial conditions
normal_bid_factor = mi.parameters['Bid_0'].value
normal_smac_factor = mi.parameters['Smac_0'].value
normal_parp_factor = mi.parameters['PARP_0'].value

bid_traj_i = traj_i.observables['mBid'] / normal_bid_factor
# TODO
# TODO

aSmac_traj_i = traj_i.observables['aSmac'] / normal_smac_factor
# TODO
# TODO

cparp_traj_i = traj_i.observables['cPARP'] / normal_parp_factor
# TODO
# TODO


In [ ]:
# plot and compare tBid

plt.plot(tspan, bid_traj_i, color='r', marker='^', label='tBID, indirect model')
# TODO
# TODO
plt.errorbar(exp_data['# Time'], exp_data['norm_IC-RP'],yerr=exp_data['nrm_var_IC-RP'] ** .5,
             ecolor='black', color='black', elinewidth=0.5, capsize=0)
plt.legend(loc=0)

In [ ]:
# plot and compare Smac
# the release of Smac is in a Snap action ([Albeck2008])
# this data displays the time in which this snap happens

# Mean and variance of Td (delay time) and Ts (switching time) of MOMP, and
# yfinal (the last value of the IMS-RP trajectory)
momp_data = np.array([9810.0, 180.0, mi.parameters['Smac_0'].value])
momp_var = np.array([7245000.0, 3600.0, 1e4])

plt.plot(tspan, aSmac_traj_i, color='r', label='aSMAC, indirect model')
# TODO
# TODO
plt.axvline(momp_data[0], -0.05, 1.05, color='black', linestyle=':',label='exp aSMAC')
plt.legend(loc=0)

In [ ]:
# plot and compare PARP

plt.plot(tspan, cparp_traj_i, color='r', marker='*', label='cPARP, indirect model')
# TODO
# TODO
plt.errorbar(exp_data['# Time'], exp_data['norm_EC-RP'],
                 yerr=exp_data['nrm_var_EC-RP'] ** .5,
                 ecolor='black', color='black', elinewidth=0.5, capsize=0)
plt.legend(loc=0)

In [ ]:
# define a function which evaluates how far the model is away from the real data
# this cost function is based on the chi^2 test
# this cost function is specifically tailored to EARM

def likelihood(position):
    param_values[rate_mask] = 10 ** position
    traj = solver.run(param_values=param_values)

    model = mi
    # normalize trajectories
    bid_traj = traj.observables['mBid'] / model.parameters['Bid_0'].value
    cparp_traj = traj.observables['cPARP'] / model.parameters['PARP_0'].value
    momp_traj = traj.observables['aSmac']

    # calculate chi^2 distance for each time course
    e1 = np.sum((exp_data['norm_IC-RP'] - bid_traj) ** 2 /
                (2 * exp_data['nrm_var_IC-RP'])) / len(bid_traj)

    e2 = np.sum((exp_data['norm_EC-RP'] - cparp_traj) ** 2 /
                (2 * exp_data['nrm_var_EC-RP'])) / len(cparp_traj)

    # Here we fit a spline to find where we get 50% release of MOMP reporter
    if np.nanmax(momp_traj) == 0:
        print('No aSmac!')
        t10 = 0
        t90 = 0
    else:
        ysim_momp_norm = momp_traj / np.nanmax(momp_traj)
        st, sc, sk = scipy.interpolate.splrep(tspan, ysim_momp_norm)
        try:
            t10 = scipy.interpolate.sproot((st, sc - 0.10, sk))[0]
            t90 = scipy.interpolate.sproot((st, sc - 0.90, sk))[0]
        except IndexError:
            t10 = 0
            t90 = 0

    # time of death  = halfway point between 10 and 90%
    td = (t10 + t90) / 2

    # time of switch is time between 90 and 10 %
    ts = t90 - t10

    # final fraction of aSMAC (last value)
    yfinal = momp_traj[-1]
    momp_sim = [td, ts, yfinal]

    e3 = np.sum((momp_data - momp_sim) ** 2 / (2 * momp_var)) / 3
    # return sum of errors ( the ',' is required)
    return e1 + e2 + e3,

In [ ]:
# load simple PSO for the particle swarm optimization
# this method re-evaluates the rate constants
# and tailors them to the real data
from simplepso.pso import PSO

# get the original parameters from the model
rate_params_i = mi.parameters_rules()
#TODO
#TODO

# the parameter set also includes initial conditions 
# those are not to be changed, so this mask filters
# purely the rates from the parameters which need to be trained
rate_mask_i = np.array([p in rate_params_i for p in mi.parameters])
#TODO
#TODO

# the original values are used as starting position for the PSO
# PSO searches in log-space for efficiency, therefore, the
# initial data is transformed into log-space
starting_position_i = np.log10(param_values_i[rate_mask_i])
#TODO
#TODO

# create a PSO object for the first model
# the parameter values to be trained are the rates from this model
param_values = param_values_i
rate_mask = rate_mask_i
solver = solver_i
pso_i = PSO(save_sampled=False, verbose=True, num_proc=4)
# use the above defined likelihood function as cost-function for the PSO
pso_i.set_cost_function(likelihood)
pso_i.set_start_position(starting_position_i)
pso_i.set_bounds(2)

# run to minimize for the best fit 
# if stopping criteria is reached, run again
pso_i.run(num_particles=25, num_iterations=50, stop_threshold=1e-5)

In [ ]:
# new rate parameters for the indirect model
display(pso_i.best)

# run indirect model with newly suggested rates
# results of PSO are in log-space -> retransform into normal space
param_values[rate_mask] = 10 ** pso_i.best

# save the new parameters for later use in an external csv file
np.savetxt("fitted_model_i_parameters.csv", param_values)

# run the indirect model with the new rate parameters
traj_i = solver_i.run(param_values=param_values)

# normalize the results again
bid_traj_i = traj_i.observables['mBid'] / normal_bid_factor
cparp_traj_i = traj_i.observables['cPARP'] / normal_parp_factor
aSmac_traj_i = traj_i.observables['aSmac'] / normal_smac_factor

In [ ]:
# do PSO for the direct model

param_values = #TODO
rate_mask = #TODO
solver = #TODO
pso_d = PSO(save_sampled=False, verbose=True, num_proc=4)
pso_d.set_cost_function(likelihood)
pso_d.set_start_position(#TODO)
pso_d.set_bounds(2)

pso_d.run(num_particles=25, num_iterations=50, stop_threshold=1e-5)

In [ ]:
#run direct model with newly suggested rates
param_values[rate_mask] = 10 ** pso_d.best
np.savetxt("fitted_model_d_parameters.csv", param_values)
traj_d = #TODO

bid_traj_d = traj_d.observables['mBid'] / normal_bid_factor
cparp_traj_d = #TODO
aSmac_traj_d = #TODO

In [ ]:
# do PSO for the embedded model

#TODO ALL THE STEPS OF PSO FOR THE EMBEDDED MODEL

In [ ]:
# run embedded model with newly suggested rates

#TODO ALL THE STEPS TO RE-RUN THE EMBEDDED MODEL WITH THE NEW RATES

In [ ]:
# compare the models using the new rates with the experimental data

plt.plot(tspan, bid_traj_i, color='r', marker='^', label='tBID, indirect model')
#TODO
#TODO
plt.errorbar(exp_data['# Time'], exp_data['norm_IC-RP'],
                 yerr=exp_data['nrm_var_IC-RP'] ** .5,
                 ecolor='black', color='black', elinewidth=0.5, capsize=0)
plt.legend(loc=0)

In [ ]:
plt.plot(tspan, aSmac_traj_i, color='r', label='aSMAC, indirect model')
#TODO
#TODO
plt.axvline(momp_data[0], -0.05, 1.05, color='black', linestyle=':',
               label='exp aSMAC')
plt.legend(loc=0)

In [ ]:
plt.plot(tspan, cparp_traj_i, color='r', label='cPARP, indirect model')
#TODO
#TODO
plt.errorbar(exp_data['# Time'], exp_data['norm_EC-RP'],
                 yerr=exp_data['nrm_var_EC-RP'] ** .5,
                 ecolor='black', color='black', elinewidth=0.5, capsize=0)
plt.legend(loc=0)